<a href="https://colab.research.google.com/github/cristobalperezp/Marketing_II/blob/main/Class_08_Introduction_to_Probabilistic_Models_and_Discrete_Time_Duration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Class_08_Introduction_to_Probabilistic_Models_and_Discrete_Time_Duration

In [2]:
#librerías para trabajo con dataframes:
import pandas as pd
import numpy as np
#librerías para graficar:
import seaborn as sns
import matplotlib.pyplot as plt
#optimizador:
from scipy.optimize import minimize
#librerías de distribuciones:
import scipy.stats as stats
import scipy.special as special

In [3]:
#cargamos las bases de datos:
!git clone https://github.com/cristobalperezp/Marketing_II.git

fatal: destination path 'Marketing_II' already exists and is not an empty directory.


In [4]:
#se lee la base de datos
mydata = pd.read_csv('/content/Marketing_II/retentiondata.txt',sep = '\t')
mydata

,year,ncust
0,0,1000
1,1,631
2,2,468
3,3,382
4,4,326
5,5,289
6,6,262
7,7,241
8,8,223
9,9,207


In [5]:
#se crea la columna 'lost'
mydata['lost'] = np.zeros(mydata.shape[0])
for i in range(mydata.shape[0]-1):
  mydata['lost'][i+1] = -(mydata['ncust'][i+1]-mydata['ncust'][i])
train = mydata.iloc[1:8]
test = mydata.iloc[9:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


##1.- loglikelihod for the shifted geometric model 

In [6]:
#loglikelihod for the shifted geometric model --------------------------------
nc = train.shape[0]
def logl(theta):
  #transform variable to impose is in the (0,1) range
  mytheta = np.exp(theta)/(1+np.exp(theta))
  #define auxiliary arrays
  pr = np.zeros(nc+1)
  ll = np.zeros(nc+1)
  #detection likelihood
  for i in range(0,nc):
    pr[i] = mytheta*( (1-mytheta)**(train['year'][i+1]-1) )
    ll[i] = train['lost'][i+1]*np.log(pr[i])
  #survival likelihood
  pr[nc] = 1-np.sum(pr[0:nc])
  ll[nc] = train['ncust'][nc]*np.log(pr[nc])
  return -np.sum(ll)

In [7]:
theta_start = 0.88

#Run the minimizer
results = minimize(logl, theta_start, method = 'BFGS')

#Print the results
mytheta = np.exp(results.x[0])/(1+np.exp(results.x[0]))
mytheta

0.22602739239009365

##2.- loglikelihod for the shifted geometric model with two segments

In [8]:
#loglikelihod for the shifted geometric model with two segments --------------
def logl2(theta):
  #transform variable to impose is in the (0,1) range
  mytheta = np.exp(theta)/(1+np.exp(theta))
      # mytheta[1]: churn probability segment 1
      # mytheta[2]: churn probability segment 2
      # mytheta[3]: probability belonging segment 1
  #define auxiliary arrays
  pr = np.zeros((2,nc+1))
  ll = np.zeros(nc+1)
  #detection likelihood
  for i in range(0,nc):
    pr[0,i] = mytheta[0]*(1-mytheta[0])**(train['year'][i+1]-1)
    pr[1,i] = mytheta[1]*(1-mytheta[1])**(train['year'][i+1]-1)
    ll[i] = train['lost'][i+1]*np.log(mytheta[2]*pr[0,i] + (1-mytheta[2])*pr[1,i])
  #survival likelihood
  pr[0,nc] = 1-np.sum(pr[0,0:nc])
  pr[1,nc] = 1-np.sum(pr[1,0:nc])
  ll[nc] = train['ncust'][nc]*np.log(mytheta[2]*pr[0,nc]+(1-mytheta[2])*pr[1,nc])
  return -np.sum(ll)

In [9]:
theta_start = [0.1,0.1,0.1]

#Run the minimizer
results = minimize(logl2, theta_start, method = 'BFGS')

#Print the results
mytheta = np.exp(results.x)/(1+np.exp(results.x))
mytheta

array([0.08302495, 0.58601148, 0.43936373])

##3.- loglikelihod for beta geometric model

In [10]:
#loglikelihod for beta geometric model ------------------------------------------------------------
def logl3(theta):
  #transform variable to impose they are positive
  myalpha = np.exp(theta[0])
  mybeta = np.exp(theta[1])
  #define auxiliary arrays
  pr = np.zeros(nc+1)
  ll = np.zeros(nc+1)
  #detection likelihood
  for i in range(0,nc):
    pr[i] = special.beta(myalpha+1,mybeta + train['year'][i+1] -1)/special.beta(myalpha,mybeta)
    ll[i] = train['lost'][i+1]*np.log(pr[i])
  #survival likelihood
  pr[nc] = 1-np.sum(pr[0:nc])
  ll[nc] = train['ncust'][nc]*np.log(pr[nc])
  return -np.sum(ll)

In [11]:
theta_start = [0.666,0.6666]

#Run the minimizer
results = minimize(logl3, theta_start, method = 'BFGS')

#Print the results
myalpha = np.exp(results.x[0])
mybeta = np.exp(results.x[1])
myalpha,mybeta

(0.7040771701889271, 1.1820427480085802)